# Motion

Luckily, the motion files are also the same across versions. It also looks like another archive with a table at the back of the file.

In [1]:
from pathlib import Path
from struct import Struct, unpack_from

ARCHIVE_FOOTER = Struct("<2I")
ARCHIVE_RECORD = Struct("<2I64s76x")


def extract_archive(data):
    offset = len(data) - ARCHIVE_FOOTER.size
    _, count = ARCHIVE_FOOTER.unpack_from(data, offset)
    for _ in range(count):
        offset -= ARCHIVE_RECORD.size  # walk the table backwards
        start, length, name = ARCHIVE_RECORD.unpack_from(data, offset)
        name = name.rstrip(b"\x00").decode("ascii")
        yield name, data[start : start + length]


data = Path("install/v1.0-us-pre/zbd/motion.zbd").read_bytes()
motions = dict(extract_archive(data))
motion_names = list(motions.keys())
print("\n".join(motion_names[:5] + motion_names[-5:]))

annihilator_collapser
vulture_trot
vulture_step
vulture_stand
vulture_run
annihilator_getupf
annihilator_getupb
annihilator_fallf
annihilator_fallb
vulture_walk


In [2]:
by_length = sorted((len(motion), name) for name, motion in motions.items())
by_length[:4]

[(850, 'elemental_stand'),
 (850, 'sunder_stand'),
 (850, 'thor_stand'),
 (992, 'cauldron_stand')]

In [3]:
from helpers import hexdump

motion = motions["thor_stand"]
hexdump(motion[:64])

00|04 00 00 00 89 88 08 3D|.......=
08|01 00 00 00 0C 00 00 00|........
16|00 00 80 BF 00 00 80 3F|.......?
24|03 00 00 00 68 69 70 0C|....hip.
32|00 00 00 13 9C FA B9 77|.......w
40|2E BD 40 87 F8 13 3F 13|..@...?.
48|9C FA B9 77 2E BD 40 87|...w..@.
56|F8 13 3F 00 00 80 3F 00|..?...?.


I see 'mech part names and the length of those names. A bit of manually reading the offsets later...

In [4]:
def name(offset):
    l, = unpack_from("<I", motion, offset)
    offset += 4
    name = motion[offset : offset + l]
    offset += l
    print(name, *unpack_from("<I", motion, offset))
    return l


l = name(24)
print(91 - 24 - l)
l = name(91)
print(160 - 91 - l)
name(160)
name(230)
name(300)
name(369)
name(438)
name(507)
name(576)
name(644)
name(712)
l = name(781)
print(781 - 712 - l)

b'hip' 12
64
b'torso' 12
64
b'lthigh' 12
b'rthigh' 12
b'rfoot' 12
b'lfoot' 12
b'rcalf' 12
b'lcalf' 12
b'larm' 12
b'rarm' 12
b'rhand' 12
b'lhand' 12
64


Seems like the motion data for each armature is 64 (or 60, disregarding the length). And there are 12 armatures:

In [5]:
unpack_from("<I4x2I8x", motion, 0)

(4, 1, 12)

In [6]:
def parse_motion(name):
    motion = motions[name]
    unk1, unk2, count = unpack_from("<I4x2I8x", motion, 0)
    print(name, unk1, unk2, count)
    offset = 24
    for _ in range(count):
        l, = unpack_from("<I", motion, offset)
        offset += 4
        part = motion[offset : offset + l].decode("ascii")
        print(part, end=" ")
        offset += l
        val, = unpack_from("<I", motion, offset)
        print(val, end=" ")
        offset += 60
    print()


parse_motion("thor_stand")
parse_motion("elemental_stand")
parse_motion("sunder_stand")
parse_motion("cauldron_stand")

thor_stand 4 1 12
hip 12 torso 12 lthigh 12 rthigh 12 rfoot 12 lfoot 12 rcalf 12 lcalf 12 larm 12 rarm 12 rhand 12 lhand 12 
elemental_stand 4 1 12
hip 12 torso 12 lthigh 12 rthigh 12 rfoot 12 lfoot 12 rcalf 12 lcalf 12 larm 12 rarm 12 rhand 12 lhand 12 
sunder_stand 4 1 12
hip 12 torso 12 lthigh 12 rthigh 12 rfoot 12 lfoot 12 rcalf 12 lcalf 12 larm 12 rarm 12 rhand 12 lhand 12 
cauldron_stand 4 1 14
hip 12 torso 12 lthigh 12 rthigh 12 rfoot 12 lfoot 12 ltoe02 12 ltoe01 12 rtoe02 12 rtoe01 12 rcalf 12 lcalf 12 larm 12 rarm 12 


Looking good so far. But it does break for some of the other ones. Presumably, there's more information in the header I need to understand.

In [7]:
for i, (name, motion) in enumerate(motions.items()):
    four, unk2, count = unpack_from("<I4x2I4x", motion, 0)
    assert four == 4
    # Avoid output spam
    if i < 5 or i > len(motions) - 5:
        print(name, unk2, count)

annihilator_collapser 60 18
vulture_trot 44 6
vulture_step 61 22
vulture_stand 99 22
vulture_run 45 22
annihilator_getupb 79 18
annihilator_fallf 22 18
annihilator_fallb 23 18
vulture_walk 56 18


`strider_stand` has an unknown value of 2, which should make investigation easier.

In [8]:
motion = motions["strider_stand"]
count = 22
hexdump(motion[:64])

00|04 00 00 00 89 88 88 3D|.......=
08|02 00 00 00 16 00 00 00|........
16|00 00 80 BF 00 00 80 3F|.......?
24|03 00 00 00 68 69 70 0C|....hip.
32|00 00 00 BD 37 86 B5 CF|....7...
40|32 BB 40 16 C3 F5 BD BD|2.@.....
48|37 86 B5 CF 32 BB 40 16|7...2.@.
56|C3 F5 BD BD 37 86 B5 CF|....7...


In [9]:
print(motion[28 : 28 + 3])
print(motion[123 : 123 + 5])
print(123 - 28 - 4 - 3)
print(motion[220 : 220 + 6])
print(220 - 123 - 4 - 5)
print(motion[318 : 318 + 6])
print(318 - 220 - 4 - 6)
print(motion[416 : 416 + 5])
print(416 - 318 - 4 - 6)

b'hip'
b'torso'
88
b'lthigh'
88
b'rthigh'
88
b'rfoot'
88


I'm suspecting that number is the number of frames in the animation. So I'll try `orion_landf` or `orion_landb` next, with 18.

In [10]:
motion = motions["orion_landf"]
count = 16
hexdump(motion[:64])

00|04 00 00 00 9A 99 19 3F|.......?
08|12 00 00 00 10 00 00 00|........
16|00 00 80 BF 00 00 80 3F|.......?
24|03 00 00 00 68 69 70 0C|....hip.
32|00 00 00 EA AF 57 BD 4B|.....W.K
40|8F EC 3F 74 34 AE C0 DC|..?t4...
48|66 6A BD 9C 8B B5 3F 82|fj....?.
56|1D AF C0 79 B1 70 BD EA|...y.p..


In [11]:
print(motion[28 : 28 + 3])
print(motion[571 : 571 + 5])
print(571 - 28 - 4 - 3)

b'hip'
b'torso'
536


If 60 is 1, and 88 is 2, then the data per frame is 28, and the base data is 32. Sure enough, 32 + 28 * 18 = 536.

In [12]:
def parse_motion(name):
    motion = motions[name]
    four, frame_count, part_count = unpack_from("<I4x2I8x", motion, 0)
    assert four == 4
    print(name, frame_count)
    offset = 24
    frame_size = 32 + 28 * frame_count
    for _ in range(part_count):
        l, = unpack_from("<I", motion, offset)
        offset += 4
        part = motion[offset : offset + l].decode("ascii")
        print(part, end=" ")
        offset += l + frame_size
    print()


parse_motion("thor_stand")
parse_motion("orion_landf")
_, longest_name = by_length[-1]
parse_motion(longest_name)

thor_stand 1
hip torso lthigh rthigh rfoot lfoot rcalf lcalf larm rarm rhand lhand 
orion_landf 18
hip torso lthigh rthigh rfoot lfoot ltoe02 ltoe01 rtoe02 rtoe01 rcalf lcalf larm rarm rhand lhand 
vulture_stand 99
hip torso lthigh rthigh rfoot lfoot ltoe02 ltoe01 ltoe03 ltoe04 rtoe03 rtoe02 rtoe01 rtoe04 rcalf rankle lcalf lankle larm rarm rhand lhand 


28 bytes of data isn't much. But then again, from decoding 3D model, translation and rotation matrices were used. A float is 32 bits or 4 bytes, and 6 * 4 = 24.

In [13]:
def parse_motion(name):
    motion = motions[name]
    four, frame_count, part_count = unpack_from("<I4x2I8x", motion, 0)
    assert four == 4
    offset = 24
    frame_size = 28 * frame_count
    parts = {}
    for _ in range(part_count):
        l, = unpack_from("<I", motion, offset)
        offset += 4
        part_name = motion[offset : offset + l].decode("ascii")
        offset += l
        header = motion[offset : offset + 32]
        offset += 32
        frames = []
        for _ in range(frame_count):
            frame = motion[offset : offset + 28]
            offset += 28
            frames.append(frame)
        parts[part_name] = (header, frames)
    return parts, frame_count


parts, _ = parse_motion("thor_stand")
for part_name, (part_header, part_frames) in parts.items():
    print(part_name)
    print(*["{:+.2f}".format(v) for v in unpack_from("<7f", part_header, 4)])
    for part_frame in part_frames:
        print(*["{:+.2f}".format(v) for v in unpack_from("<7f", part_frame, 0)])

hip
-0.00 +5.91 +0.58 -0.00 +5.91 +0.58 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
torso
-0.00 +0.90 +0.21 -0.00 +0.90 +0.21 +1.00
+0.00 +0.00 -0.00 +1.00 +0.00 +0.00 -0.00
lthigh
-1.19 +0.60 +0.29 -1.19 +0.60 +0.29 -0.99
-0.11 -0.00 -0.00 -0.99 -0.11 -0.00 -0.00
rthigh
+1.19 +0.60 +0.29 +1.19 +0.60 +0.29 -1.00
-0.05 +0.00 +0.00 -1.00 -0.05 +0.00 +0.00
rfoot
+1.69 -4.53 +0.42 +1.69 -4.53 +0.42 +1.00
-0.00 +0.00 -0.00 +1.00 -0.00 +0.00 -0.00
lfoot
-1.69 -4.52 -0.29 -1.69 -4.52 -0.29 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
rcalf
+0.51 -3.51 +0.02 +0.51 -3.51 +0.02 -0.98
+0.19 -0.00 -0.00 -0.98 +0.19 -0.00 -0.00
lcalf
-0.50 -3.51 +0.01 -0.50 -3.51 +0.01 -0.98
+0.17 -0.00 -0.00 -0.98 +0.17 -0.00 -0.00
larm
-2.00 +2.30 +0.25 -2.00 +2.30 +0.25 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
rarm
+1.79 +2.32 +0.11 +1.79 +2.32 +0.11 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
rhand
+1.01 -0.76 +0.00 +1.01 -0.76 +0.00 +1.00
+0.00 +0.00 +0.00 +1.00 +0.00 +0.00 +0.00
lhand
-0.71 

In [14]:
parts, frame_count = parse_motion("thor_walk")
for part_name, (part_header, part_frames) in parts.items():
    i, = unpack_from("<I", part_header, 0)
    print(part_name, i, frame_count)
    print(*["{:+.2f}".format(v) for v in unpack_from("<7f", part_header, 4)])
    # Commented out to avoid output spam
    # for part_frame in part_frames:
    #    print(*["{:+.2f}".format(v) for v in unpack_from("<7f", part_frame, 0)])

hip 12 56
+0.09 +5.73 -3.28 +0.11 +5.72 -3.42 +0.12
torso 12 56
-0.00 +0.90 +0.21 -0.00 +0.90 +0.21 -0.00
lthigh 12 56
-1.19 +0.60 +0.29 -1.19 +0.60 +0.29 -1.19
rthigh 12 56
+1.19 +0.60 +0.29 +1.19 +0.60 +0.29 +1.19
rfoot 12 56
+1.67 -4.27 -1.47 +1.65 -4.30 -1.35 +1.63
lfoot 12 56
-1.91 -4.33 +1.82 -1.91 -4.30 +1.85 -1.89
rcalf 12 56
+0.50 -3.47 -0.01 +0.50 -3.47 -0.01 +0.50
lcalf 12 56
-0.50 -3.51 +0.01 -0.50 -3.51 +0.01 -0.50
larm 12 56
-2.00 +2.30 +0.25 -2.00 +2.30 +0.25 -2.00
rarm 12 56
+1.79 +2.32 +0.11 +1.79 +2.32 +0.11 +1.79
rhand 12 56
+1.01 -0.76 +0.00 +1.01 -0.76 +0.00 +1.01
lhand 12 56
-0.71 -1.15 -0.00 -0.71 -1.15 -0.00 -0.71


Hmm, I think I can see a cyclical pattern of 3 values. It's too consistent to be a coincidence. In fact, there's a pattern: 57 rows of triplets, and then 57 rows of quadruplets:

```
+0.09 +5.73 -3.28
+0.11 +5.72 -3.42
+0.12 +5.72 -3.56
[...]
+0.05 +5.74 -10.98
+0.07 +5.74 -11.12
+0.09 +5.73 -3.28
```

```
+1.00 +0.00 +0.08 +0.00
+1.00 -0.00 +0.08 -0.00
+1.00 -0.00 +0.08 -0.00
[...]
+1.00 +0.00 +0.07 +0.01
+1.00 +0.00 +0.08 +0.01
+1.00 +0.00 +0.08 +0.00
```

I'm guessing the triplets are location data, and the quadruplets are rotation. For one, because I dumped the location data when exporting the models, and it's somewhat close.

The rotation data took way longer to figure out, over a month. In the models, rotation is encoded as three rotational angles, aka. Euler angles. However, here, through much trial and error, I discovered that the rotation is encoded using [quaternions](https://en.wikipedia.org/wiki/Quaternion). Quaternions are a great way of encoding rotation that doesn't suffer from [gimbal lock](https://en.wikipedia.org/wiki/Gimbal_lock), and computationally cheap compared to matrices. But it was still a surprise to see them used here.

Even with this revelation, there are still issues. For the Mad Cat, the arms are displaced from the torso. For the Vulture, it's quite "exploded". For the Orion, the toes are displaced.

## Next up

[`Mech3Msg.dll` message table/translations extraction](09-mech3msg.ipynb)

**WARNING**: Work in progress below, trying to figure out why some animations don't apply properly

In [19]:
def parse_motion(motion):
    four, unk, frame_count, part_count, minus_one, plus_one = unpack_from("<If2I2f", motion, 0)
    assert four == 4
    assert minus_one == -1.0
    assert plus_one == 1.0
    assert unk > 0.0
    offset = 24
    frame_count += 1
    parts = {}
    for _ in range(part_count):
        # name size, name, twelve?
        name_size, = unpack_from("<I", motion, offset)
        offset += 4
        part_name = motion[offset : offset + name_size].decode("ascii")
        offset += name_size
        twelve, = unpack_from("<I", motion, offset)
        assert twelve == 12
        offset += 4
        # location
        location_count = frame_count * 3
        location_data = unpack_from(f"<{location_count}f", motion, offset)
        location_values = [location_data[i:i+3] for i in range(0, location_count, 3)]
        offset += location_count * 4
        # rotation
        rotation_count = frame_count * 4
        rotation_data = unpack_from(f"<{rotation_count}f", motion, offset)
        rotation_values = [rotation_data[i:i+4] for i in range(0, rotation_count, 4)]
        offset += rotation_count * 4
        parts[part_name] = list(zip(location_values, rotation_values))
    return parts

parsed = {
    name: parse_motion(motion)
    for name, motion in motions.items()
}